In [1]:
!pip install transformers

In [2]:
!pip install tabulate

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [4]:
from tqdm import tqdm, tqdm_notebook

In [5]:
# file_path = './drive/MyDrive/movie_reviews.xlsx'
file_path = './movie_reviews.xlsx'
df = pd.read_excel(file_path)
print(df.dtypes)
df.head()

label       int64
url        object
rating      int64
date       object
content    object
dtype: object


,label,url,rating,date,content
0,0,/review/rw1142613/,6,4 August 2005,"It's an ""emotional manipulation for dummies"" t..."
1,0,/review/rw1587268/,6,30 January 2007,"Tim Robbins plays Andy Dufresne, a man convict..."
2,0,/review/rw0349436/,6,18 December 2003,It is the most basic of all principles of film...
3,0,/review/rw0349359/,6,30 July 2003,Little more than average drama for popular tas...
4,0,/review/rw0348197/,6,10 September 1998,An UNDER-RATED movie?? Gimme a break! This is ...


In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to C:\Users\Youngchan
[nltk_data]     Kim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
'''1. 전체 문장'''
# text = df.content.values
# labels = df.label.values

'1. 전체 문장'

In [8]:
%%time
'''2. 두문장 씩 묶어서'''
text = []
labels = []
content = df.content.values
temp = df.label.values
for i, sentence in enumerate(content):
    if temp[i] >= 50: 
        break
    try:
        # print(content)
        sentences = sent_tokenize(str(sentence))
        for j in range(0, len(sentences)):
            pair = sentences[j]+sentences[j + 1] if j + 1 < len(sentences) else sentences[j]
            text.append(pair)
            labels.append(temp[i])
    except Exception as e:
        print(e)
        print(content)

Wall time: 3.97 s


불용어 제거
```
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

example = "Family is not an important thing. It's everything."
stop_words = set(stopwords.words('english')) 

word_tokens = word_tokenize(example)

result = []
for word in word_tokens: 
    if word not in stop_words: 
        result.append(word) 
```

길이가 짧은 단어 삭제
```
import re
text = "I was wondering if anyone out there could enlighten me on this car."

# 길이가 1~2인 단어들을 정규 표현식을 이용하여 삭제
shortword = re.compile(r'\W*\b\w{1,2}\b')
print(shortword.sub('', text))
```

In [9]:
len(text)

148367

In [10]:
len(labels)

148367

In [11]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True # True/모두 소문자로 변환. False: 대소문자 구분
    )

bert-base-uncased  
bert-base-cased  
bert-base-multilingual-uncased  
bert-base-multilingual-cased  


In [12]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                      tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table, headers = ['Tokens', 'Token IDs'], tablefmt = 'fancy_grid'))

print_rand_sentence()

The script is well-written, and the cinematography and sound design are top-notch.This movie is a must-see for any war movie enthusiast, as well as anyone who appreciates masterful filmmaking.Rating: 9/10Quote: "Earn this."- Captain John Miller (Tom Hanks)
╒════════════════╤═════════════╕
│ Tokens         │   Token IDs │
╞════════════════╪═════════════╡
│ the            │        1996 │
├────────────────┼─────────────┤
│ script         │        5896 │
├────────────────┼─────────────┤
│ is             │        2003 │
├────────────────┼─────────────┤
│ well           │        2092 │
├────────────────┼─────────────┤
│ -              │        1011 │
├────────────────┼─────────────┤
│ written        │        2517 │
├────────────────┼─────────────┤
│ ,              │        1010 │
├────────────────┼─────────────┤
│ and            │        1998 │
├────────────────┼─────────────┤
│ the            │        1996 │
├────────────────┼─────────────┤
│ cinematography │       16434 │
├────────────────

In [13]:
def print_sentence(index):
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                      tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table, headers = ['Tokens', 'Token IDs'], tablefmt = 'fancy_grid'))

print_sentence(0)

It's an "emotional manipulation for dummies" type of film, with boring direction and cardboard characters.Nice performances by the actors though, some great moments, but nothing outside the "Hallmark meets Stephen King meets Prison" feel of the entire movie.I saw it one and a half times, and would see it again only if I got paid for the trouble.Similarly, I hated 'The Green Mile", only later did I find out it's the work of Darabont.My recommendation: Skip this, and go see OZ again.
╒════════════════╤═════════════╕
│ Tokens         │   Token IDs │
╞════════════════╪═════════════╡
│ it             │        2009 │
├────────────────┼─────────────┤
│ '              │        1005 │
├────────────────┼─────────────┤
│ s              │        1055 │
├────────────────┼─────────────┤
│ an             │        2019 │
├────────────────┼─────────────┤
│ "              │        1000 │
├────────────────┼─────────────┤
│ emotional      │        6832 │
├────────────────┼─────────────┤
│ manipulation   │

In [14]:
%%time
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 128,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])

token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Wall time: 2min 23s


In [15]:
token_id[1]

tensor([  101,  3835,  4616,  2011,  1996,  5889,  2295,  1010,  2070,  2307,
         5312,  1010,  2021,  2498,  2648,  1996,  1000, 25812,  6010,  4459,
         2332,  6010,  3827,  1000,  2514,  1997,  1996,  2972,  3185,  1012,
         1045,  2387,  2009,  2028,  1998,  1037,  2431,  2335,  1010,  1998,
         2052,  2156,  2009,  2153,  2069,  2065,  1045,  2288,  3825,  2005,
         1996,  4390,  1012,  6660,  1010,  1045,  6283,  1005,  1996,  2665,
         3542,  1000,  1010,  2069,  2101,  2106,  1045,  2424,  2041,  2009,
         1005,  1055,  1996,  2147,  1997, 18243,  7875, 12162,  1012,  2026,
        12832,  1024, 13558,  2023,  1010,  1998,  2175,  2156, 11472,  2153,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [16]:
%%time
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, headers = ['Tokens', 'Token IDs', 'Attention Mask'], tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ [CLS]         │         101 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ it            │        2009 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ '             │        1005 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ s             │        1055 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ not           │        2025 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ entertainment │        4024 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ ,             │        1010 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ it            │        2009 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ '             │        1005 │

In [17]:
val_ratio = 0.25
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [18]:
def b_tp(preds, labels):
    '''Returns True Positives (TP): count of correct predictions of actual class 1'''
    return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
    '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
    return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
    '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
    return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
    '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
    return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
    '''
    Returns the following metrics:
        - accuracy    = (TP + TN) / N
        - precision   = TP / (TP + FP)
        - recall      = TP / (TP + FN)
        - specificity = TN / (TN + FP)
    '''
    preds = np.argmax(preds, axis = 1).flatten()
    labels = labels.flatten()
    tp = b_tp(preds, labels)
    tn = b_tn(preds, labels)
    fp = b_fp(preds, labels)
    fn = b_fn(preds, labels)
    b_accuracy = (tp + tn) / len(labels)
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    return b_accuracy, b_precision, b_recall, b_specificity

In [19]:
%%time
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 50,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Wall time: 1.41 s


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
%%time
# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 3

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(tqdm_notebook(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:   0%|                                                                                                                                                                                                                      | 0/3 [00:00<?, ?it/s]<timed exec>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/6955 [00:00<?, ?it/s]

Epoch:  33%|███████████████████████████████████████████████████████████████████▎                                                                                                                                      | 1/3 [54:52<1:49:45, 3292.85s/it]


	 - Train loss: 1.4259
	 - Validation Accuracy: 0.0167
	 - Validation Precision: 0.4444
	 - Validation Recall: 0.8578
	 - Validation Specificity: 0.2131



  0%|          | 0/6955 [00:00<?, ?it/s]

Epoch:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                   | 2/3 [1:49:09<54:31, 3271.77s/it]


	 - Train loss: 0.8270
	 - Validation Accuracy: 0.0184
	 - Validation Precision: 0.5963
	 - Validation Recall: 0.8767
	 - Validation Specificity: 0.2844



  0%|          | 0/6955 [00:00<?, ?it/s]

Epoch: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [2:45:01<00:00, 3300.41s/it]


	 - Train loss: 0.5374
	 - Validation Accuracy: 0.0196
	 - Validation Precision: 0.6159
	 - Validation Recall: 0.9546
	 - Validation Specificity: 0.3642

Wall time: 2h 45min 1s


In [22]:
def predict(sentence):
    # We need Token IDs and Attention Mask for inference on the new sentence
    test_ids = []
    test_attention_mask = []

    # Apply the tokenizer
    encoding = preprocessing(new_sentence, tokenizer)

    # Extract IDs and Attention Mask
    test_ids.append(encoding['input_ids'])
    test_attention_mask.append(encoding['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)

    # Forward pass, calculate logit predictions
    with torch.no_grad():
      output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

    prediction = np.argmax(output.logits.cpu().numpy()).flatten().item()

    print('Input Sentence: ', new_sentence)
    print('Predicted Class: ', prediction)


new_sentence = 'a fantasy movie'
predict(new_sentence)

Input Sentence:  a fantasy movie
Predicted Class:  12


C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
PATH = './' 
torch.save(model, PATH + 'BERT_movie50_1.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict50_1.pt')  # 모델 객체의 state_dict 저장